In [7]:
import os
import json

# Project root directory
root_dir = "/Users/prisriva/Desktop/fai_final"

# Dataset root
data_dir = os.path.join(root_dir, "data", "raw")

# Split directories
train_dir = os.path.join(data_dir, "train")
val_dir   = os.path.join(data_dir, "val")
test_dir  = os.path.join(data_dir, "test")

# Image folders
train_img_dir = os.path.join(train_dir, "images")
val_img_dir   = os.path.join(val_dir, "images")
test_img_dir  = os.path.join(test_dir, "images")

# Annotation files
train_ann = os.path.join(train_dir, "annotations.json")
val_ann   = os.path.join(val_dir, "annotations.json")
test_ann  = os.path.join(test_dir, "annotations.json")

train_ann, val_ann, test_ann


('/Users/prisriva/Desktop/fai_final/data/raw/train/annotations.json',
 '/Users/prisriva/Desktop/fai_final/data/raw/val/annotations.json',
 '/Users/prisriva/Desktop/fai_final/data/raw/test/annotations.json')

In [8]:
def load_coco_json(path):
    with open(path, "r") as f:
        return json.load(f)

train_data = load_coco_json(train_ann)
val_data   = load_coco_json(val_ann)
test_data  = load_coco_json(test_ann)

print("Loaded:")
print(" Train:", len(train_data["images"]), "images,", len(train_data["annotations"]), "annotations")
print(" Val:  ", len(val_data["images"]),   "images,", len(val_data["annotations"]),   "annotations")
print(" Test: ", len(test_data["images"]),  "images,", len(test_data["annotations"]),  "annotations")


Loaded:
 Train: 2816 images, 6211 annotations
 Val:   810 images, 1744 annotations
 Test:  374 images, 785 annotations


In [5]:
def cardd_detection_collate_fn(batch):
    """
    For detection models:
    batch = [(image, target), (image, target), ...]

    Return:
        images: list[Tensor]
        targets: list[dict]
    """
    return tuple(zip(*batch))


In [6]:
def load_cardd_split(root_dir: str, split: str, transforms=None):
    """
    Loads one of the dataset splits:
        split in {"train", "val", "test"}

    Assumes directory structure:
        {root_dir}/{split}/images/
        {root_dir}/{split}/annotations.json
    """
    images_dir = os.path.join(root_dir, split, "images")
    ann_path = os.path.join(root_dir, split, "annotations.json")

    if not os.path.exists(images_dir):
        raise FileNotFoundError(f"Images folder not found: {images_dir}")

    if not os.path.exists(ann_path):
        raise FileNotFoundError(f"Annotation file not found: {ann_path}")

    return CarDDDetectionDataset(images_dir, ann_path, transforms)


In [9]:
def verify_image_files(coco_data, img_dir, split_name):
    json_images = coco_data["images"]

    missing = []

    for img_entry in json_images:
        fname = img_entry["file_name"]
        path = os.path.join(img_dir, fname)

        if not os.path.exists(path):
            missing.append(fname)

    print(f"\n--- {split_name.upper()} REPORT ---")
    print(f"Images listed in JSON : {len(json_images)}")
    print(f"Missing image files   : {len(missing)}")

    if missing:
        print("\nExamples of missing files:")
        for m in missing[:10]:
            print("   -", m)

    return missing

missing_train = verify_image_files(train_data, train_img_dir, "train")
missing_val   = verify_image_files(val_data,   val_img_dir,   "val")
missing_test  = verify_image_files(test_data,  test_img_dir,  "test")



--- TRAIN REPORT ---
Images listed in JSON : 2816
Missing image files   : 0

--- VAL REPORT ---
Images listed in JSON : 810
Missing image files   : 0

--- TEST REPORT ---
Images listed in JSON : 374
Missing image files   : 0


In [10]:
from collections import defaultdict

def analyze_coco_annotations(coco_data, split_name):
    anns = coco_data["annotations"]

    cls_counts = defaultdict(int)

    for ann in anns:
        cls_counts[ann["category_id"]] += 1

    print(f"\n--- {split_name.upper()} ANNOTATION STATS ---")
    print("Total annotations:", len(anns))
    print("Class distribution:")

    for cls_id, count in sorted(cls_counts.items()):
        print(f"  Class {cls_id}: {count}")

    # bbox sanity checks
    bad_boxes = []
    for ann in anns:
        x, y, w, h = ann["bbox"]
        if w <= 0 or h <= 0:
            bad_boxes.append(ann["id"])

    print("Bad boxes:", len(bad_boxes))
    return cls_counts, bad_boxes

train_stats = analyze_coco_annotations(train_data, "train")
val_stats   = analyze_coco_annotations(val_data,   "val")
test_stats  = analyze_coco_annotations(test_data,  "test")



--- TRAIN ANNOTATION STATS ---
Total annotations: 6211
Class distribution:
  Class 1: 1806
  Class 2: 2560
  Class 3: 651
  Class 4: 475
  Class 5: 494
  Class 6: 225
Bad boxes: 0

--- VAL ANNOTATION STATS ---
Total annotations: 1744
Class distribution:
  Class 1: 501
  Class 2: 728
  Class 3: 177
  Class 4: 135
  Class 5: 141
  Class 6: 62
Bad boxes: 0

--- TEST ANNOTATION STATS ---
Total annotations: 785
Class distribution:
  Class 1: 236
  Class 2: 307
  Class 3: 70
  Class 4: 71
  Class 5: 69
  Class 6: 32
Bad boxes: 0


In [11]:
def extract_image_to_classes(coco_data):
    mapping = defaultdict(set)

    for ann in coco_data["annotations"]:
        img_id = ann["image_id"]
        cls_id = ann["category_id"]
        mapping[img_id].add(cls_id)

    # convert sets to lists to match your classification preprocessing
    final_map = {k: sorted(list(v)) for k, v in mapping.items()}
    return final_map


train_img_classes = extract_image_to_classes(train_data)
val_img_classes   = extract_image_to_classes(val_data)
test_img_classes  = extract_image_to_classes(test_data)

print("\nExample (train image -> classes):")
for k in list(train_img_classes.keys())[:5]:
    print(k, ":", train_img_classes[k])



Example (train image -> classes):
1 : [2, 6]
2 : [6]
3 : [6]
4 : [6]
5 : [6]


In [12]:
example_id = list(train_img_classes.keys())[0]

print("\nExample image_id:", example_id)
print("File name:", next(img["file_name"] for img in train_data["images"] if img["id"] == example_id))
print("Classes:", train_img_classes[example_id])

# Show boxes
print("\nBounding boxes:")
for ann in train_data["annotations"]:
    if ann["image_id"] == example_id:
        print("  bbox:", ann["bbox"], "class:", ann["category_id"])



Example image_id: 1
File name: 000001.jpg
Classes: [2, 6]

Bounding boxes:
  bbox: [167.04, 40.21, 202.79, 131.34] class: 2
  bbox: [160.66, 112.45, 684.19, 551.02] class: 6
